<a href="https://colab.research.google.com/github/saberforce/Portfolio-Component-Risk-Return-Landscape/blob/main/Portfolio_Tracking_Error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pycoingecko


In [2]:
import pandas as pd
import requests
import plotly.express as px

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()


In [210]:
def get_asset_price(assetlist, days):
  # For Each Coin Get A List Of Prices Over The Last yyy Days
  assetdict={}
  for coin in assetlist:
    asset_price=cg.get_coin_market_chart_by_id(id=coin,vs_currency='usd',days=days)
    pricelist=[]
    num_of_prices=len(asset_price['prices'])
    for k in range(num_of_prices):

      coinprice=(asset_price['prices'][k][1])
      pricelist.append(coinprice)

    #Prep data - some coins have deficient data count
    assetdict[f'{coin}']=pd.Series(pricelist)

  return assetdict


def asset_price_change(assetdict, assetlist, assetlistportion):
  # Getting A List of Price Change Percentage (or Returns Percentage) 
  # Derived From Daily Prices Of Each Coin

  df1=pd.DataFrame()
  n=0
  price_change = [x+"_price_change" for x in assetlist]

  for k,v in assetdict.items():
    df1[price_change[n]]=(v.pct_change(1))*assetlistportion[n]
    n=n+1
  return df1


def plot_line(days, error, dfa, x, y, labels={}, title=''):
  #Plotting A Line Graph For Y Variable(s) On A Chart With 2 Fixed Horizontal Lines
  fig = px.line(dfa, x, y,
                labels=labels,
                title=title
                )
 
  fig.update_traces(line_color='green', line_width=2)
  fig.add_hline(y=3.0, y1=3.0, annotation_text="3%", annotation_position="top left",
                line_width=1.5, line_dash="dash", line_color="blue")
  fig.add_hline(y=-3, y1=-3, annotation_text="-3%", annotation_position="bottom left",
                line_width=1.5, line_dash="dash", line_color="red")
  fig.add_annotation(
    text = (f'{days} Days Period Tracking Error % is {error}')
    ,showarrow=False
    , x = 0
    , y = -0.15
    , xref='paper'
    , yref='paper' 
    , font=dict(size=20, color="blue")
    , align="left"
    )

  fig.show()



In [213]:
def main():

  pd.set_option('display.max_columns', None)

# SPROUT PORTFOLIO

  #Portfolio's Underlying Assets
  portfoliolist=['bitcoin', 'ethereum', 'wmatic','chainlink', 'uniswap','the-sandbox',
                 'decentraland','the-graph','aave', 'wrapped-usdc']
       

  #Allocated Asset % Inside The Portfolio In Ordered Sequence
  portfoliolistportion=[0.2,0.2,0.1,0.1,0.05,0.05,0.05,0.05,0.05,0.15]

  #Period Assessed (In Days)
  days=180

  #Deriving All Prices For Each Portfolio Asset Over No. Of Days Assessed
  #Deriving All Price Changes For Each Portfolio Asset Inside A Dataframe
  #Deriving Change In Value of Portfolio From Price Change of Each Underlying Asset
  portfoliodict=get_asset_price(portfoliolist, days)
  
  df_portfolio_asset_price_change= asset_price_change(portfoliodict, 
                                                      portfoliolist, 
                                                      portfoliolistportion)

  df_portfolio_asset_price_change['Portfolio_value_change']=df_portfolio_asset_price_change.sum(axis=1)
  #print("this is df1", df_portfolio_asset_price_change)


# BENCHMARK # 1 (Crypto20)

  benchmarklist=['crypto20']
       
  # Allocated % For Each Benchmark In Ordered Sequence
  
  benchmarklistportion=[1.0]

  #Deriving All Prices For Benchmark Over No. Of Days Assessed
  #Deriving All Price Changes For Benchmark Inside A Dataframe

  c20_dict=get_asset_price(benchmarklist, days)
  
  df_c20_price_change= asset_price_change(c20_dict, benchmarklist, benchmarklistportion)

  #print(df_c20_price_change)

# TRACKING ERROR IN %
  
  # Finalizing The Dataframe For Portfolio vs Benchmark

  finallist = [df_portfolio_asset_price_change['Portfolio_value_change']*100, 
                  df_c20_price_change['crypto20_price_change']*100]

  headers=["Portfolio returns (daily %)","crypto20 returns (daily %)"]

  
  df_final = pd.concat(finallist, axis=1, keys=headers)


  # Finalizing The Tracking Error % On Daily Basis And For The Full Period Assessed

  df_final['Daily Tracking Error %']=df_final["Portfolio returns (daily %)"]-df_final["crypto20 returns (daily %)"]

  full_period_tracking_error=round(df_final['Daily Tracking Error %'].std(), 3)
 
  print(df_final)
  print("Tracking Error % For Full Period Assessed", full_period_tracking_error)


# Plot Line Chart of Tracking Error

  plot_line(days,full_period_tracking_error,df_final, x=df_final.index, 
            y=['Daily Tracking Error %'], 
            labels={'index': f'No. of Days ({days})', 'value': 'Error %', 
                    'variable': 'Legend'}, 
            title="Daily Tracking Error % (Portfolio Vs Crypto20)")

if __name__ == "__main__":
  main()

     Portfolio returns (daily %)  crypto20 returns (daily %)  \
0                       0.000000                         NaN   
1                       3.551220                    3.536452   
2                       4.469519                    5.951696   
3                       0.311197                    0.493248   
4                       0.875831                   -0.688333   
..                           ...                         ...   
176                    -4.267838                   -5.095454   
177                    -5.003188                   -5.469113   
178                    -0.491274                    0.208507   
179                    -1.048780                    0.279638   
180                    -1.171608                   -2.069604   

     Daily Tracking Error %  
0                       NaN  
1                  0.014768  
2                 -1.482177  
3                 -0.182051  
4                  1.564164  
..                      ...  
176                0.